In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/medium-articles/medium_articles.csv


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder


# load dataset

In [3]:
data_dir = '/kaggle/input/medium-articles'
data_file = 'medium_articles.csv'
to_file = os.path.join(data_dir, data_file)
df = pd.read_csv(to_file)

# **understanding data**

In [4]:
df.head(10)

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."
5,"Mentally, Young Adults Are Suffering Most From...","Mentally, Young Adults Are Suffering Most From...",https://medium.com/the-partnered-pen/mentally-...,['Ryan Fan'],2020-11-19 15:27:36.001000+00:00,"['Society', 'Mental Health', 'Health', 'Nonfic..."
6,How to Turn Your Popular Blog Series Into a Be...,How to Turn Your Popular Blog Series Into a Be...,https://frankmckinley.medium.com/want-to-turn-...,['Frank Mckinley'],2020-01-28 03:36:58.566000+00:00,"['Books', 'Entrepreneurship', 'Writing', 'Mark..."
7,Dr Faisal Dar — Pioneer of Liver Transplantati...,Dr Faisal Dar — Pioneer of Liver Transplantati...,https://medium.com/storyfest/dr-faisal-dar-pio...,['Fatima Arif'],2019-02-28 06:01:04.914000+00:00,"['People', 'Storyfest', 'Health', 'Pakistan', ..."
8,Sunlight — The Natural Supplement For Our Ment...,Sunlight — The Natural Supplement For Our Ment...,https://medium.com/wholistique/sunlight-the-na...,['Jerren Gan'],2020-12-16 10:47:37.171000+00:00,"['Self Improvement', 'Mental Health', 'Health'..."
9,Occam’s dice,Occam’s dice\n\nDistrusting biological metapho...,https://medium.com/the-spike/occams-dice-258aa...,['Kelly Clancy'],2018-08-22 14:55:56.262000+00:00,"['Machine Learning', 'Science', 'Neuroscience'..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192368 entries, 0 to 192367
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      192363 non-null  object
 1   text       192368 non-null  object
 2   url        192368 non-null  object
 3   authors    192368 non-null  object
 4   timestamp  192366 non-null  object
 5   tags       192368 non-null  object
dtypes: object(6)
memory usage: 8.8+ MB


In [6]:
df.drop(columns = ["url", "authors", "timestamp"], inplace = True)

In [7]:
df.drop_duplicates(keep = "first", inplace = True)
df.reset_index(inplace = True, drop = True)

In [8]:
df = df.head(5000)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   5000 non-null   object
 1   text    5000 non-null   object
 2   tags    5000 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB


# handeling missing data

In [10]:
max_non_null_val = df["text"].count()
print("Percentage of missing values in each column:\n")
missing_percent = []
for col in df.columns:
  missing_percent.append(df[col].isnull().sum()/max_non_null_val*100)
  print(f"{col} : {df[col].isnull().sum()/max_non_null_val*100}%")

Percentage of missing values in each column:

title : 0.0%
text : 0.0%
tags : 0.0%


In [11]:
df.dropna(inplace = True)
df.reset_index(drop=True, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   5000 non-null   object
 1   text    5000 non-null   object
 2   tags    5000 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB


#Loading the Spacy model


In [12]:
!pip install spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 28.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [13]:
import spacy
nlp = spacy.load("en_core_web_md")

##preprocessing the data

In [14]:
df["text"] = df["title"] + " " + df["text"] + " " + df["tags"]

In [15]:
def preprocess_text(text):
      doc = nlp(text)
      tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha]
      review = ' '.join(tokens)
      return review


In [16]:
corpus = df['text'].apply(preprocess_text)

In [17]:
# corpus = []
# for i in range(0, df["text"].count()):
#     review = df['text'][i]
#     doc = nlp(review)
#     tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha]
#     review = ' '.join(tokens)
#     corpus.append(review)

In [18]:
# indexes = []

# for index, sentence in enumerate(corpus):
#     if sentence.strip() == "":
#         print(index)
#         print(df["text"][index])
#         indexes.append(index)

# df.drop(index=indexes, inplace=True)
# df.reset_index(drop=True, inplace=True)

In [19]:
# for i in indexes:
#   corpus.pop(i)

# Creating bag of words model

In [115]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()

##Scaling the data

In [107]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler(with_mean = False)
# X = scaler.fit_transform(X)

##Making an cosine similarity matrix

In [100]:
# from sklearn.metrics.pairwise import cosine_similarity

# matrix = cosine_similarity(X,X)
# print(matrix)
# print(matrix.flatten())

In [116]:
from sklearn.metrics.pairwise import cosine_similarity
def get_similar_article(text, vectorizer, X, df, top_n=5):
    preprocessed_text = preprocess_text(text)
    X_text = vectorizer.transform([preprocessed_text])
    similarity_scores = cosine_similarity(X, X_text).flatten()
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    return df.iloc[top_indices], similarity_scores[top_indices], similarity_scores

In [117]:
similar_articles, scores, similarity_scores = get_similar_article(df["text"][0],cv, X, df)
print(scores)
similar_articles

[1.         0.42523619 0.42173792 0.38316829 0.37724186]


,title,text,tags
0,Mental Note Vol. 24,Mental Note Vol. 24 Photo by Josh Riemer on Un...,"['Mental Health', 'Health', 'Psychology', 'Sci..."
496,Let go of these things for a happier 2021,Let go of these things for a happier 2021 by: ...,"['Nonfiction', 'Psychology', 'Relationships', ..."
4221,8 Life Lessons I’ve Learned at 40-Something Th...,8 Life Lessons I’ve Learned at 40-Something Th...,"['Life Lessons', 'Self Love', 'Aging', 'Self I..."
220,Why Do I Feel Guilty When My Mental Health is ...,Why Do I Feel Guilty When My Mental Health is ...,"['Creativity', 'Mindfulness', 'Mental Health',..."
2294,Are You Living Your Life with the End Result i...,Are You Living Your Life with the End Result i...,"['Motivation', 'Self', 'Psychology', 'Life Les..."


# Creating TF-IDF model

In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()
X = tv.fit_transform(corpus).toarray()

##Scaling the data

In [104]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler(with_mean = False)
# X = scaler.fit_transform(X)

In [114]:
similar_articles, scores, similarity_scores = get_similar_article(df["text"][0], tv, X, df)
print(scores)
similar_articles

[1.         0.20868122 0.20314881 0.19685895 0.19384693]


,title,text,tags
0,Mental Note Vol. 24,Mental Note Vol. 24 Photo by Josh Riemer on Un...,"['Mental Health', 'Health', 'Psychology', 'Sci..."
2289,Don’t Try to be Anyone’s Rock if You’re a Pile...,Don’t Try to be Anyone’s Rock if You’re a Pile...,"['Mindfulness', 'Mental Health', 'Self Love', ..."
2627,Asshole Astrology: Week of 21 December 2020,Asshole Astrology: Week of 21 December 2020 As...,"['Relationships', 'Self', 'Psychology', 'Life ..."
4629,Christmas,Christmas Christmas\n\nWish\n\nPhoto by loly g...,"['Christmas', 'Nonfiction', 'Family', 'Spiritu..."
1982,Bounce! A Therapist’s New Year Resolution for ...,Bounce! A Therapist’s New Year Resolution for ...,"['Resilience', 'Mental Health', 'Stress', 'Cor..."
